In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models


In [ ]:
from google.colab import files
files.upload()


In [ ]:
import numpy as np
data=np.load("candidate_dataset.npz")
print(data.files)

In [ ]:
x_train=data["x_train"]
y_train=data["y_train"]
x_val=data["x_val"]
y_val=data["y_val"]

In [ ]:
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

In [ ]:
y_train = y_train.squeeze()
y_val   = y_val.squeeze()

In [ ]:
print(y_train.shape, y_val.shape)


In [ ]:
x_train = x_train / 255.0
x_val   = x_val / 255.0


In [ ]:
print(x_train.min(), x_train.max())


In [ ]:
classes, counts = np.unique(y_train, return_counts=True)
plt.bar(classes, counts)
plt.xlabel("Class")
plt.ylabel("Number of Samples")
plt.title("Training Class Distribution (Noisy Labels)")
plt.show()

In [ ]:
classes, counts = np.unique(y_train, return_counts=True)
plt.bar(classes, counts)
plt.title("Training Class Distribution (Noisy Labels)")
plt.xlabel("Class")
plt.ylabel("Count")
plt.show()

plt.figure(figsize=(6,6))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(x_train[i])
    plt.title(f"Label: {y_train[i]}")
    plt.axis("off")
plt.show()

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), padding="same", activation="relu", input_shape=(28,28,3)),
    layers.BatchNormalization(),

    layers.Conv2D(64, (3,3), padding="same", activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.30),

    layers.Conv2D(128, (3,3), padding="same", activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.40),

    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.50),

    layers.Dense(7, activation="softmax")
])


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=4,
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=2,
    verbose=1
)

history = model.fit(
    x_train, y_train,
    epochs=30,
    batch_size=64,
    validation_data=(x_val, y_val),
    callbacks=[early_stop, reduce_lr],
    verbose=1
)


In [ ]:
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Val Loss")
plt.legend()
plt.show()

plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Val Accuracy")
plt.legend()
plt.show()


In [ ]:
model.save("dermatology_cnn_model.h5")
print("Model saved successfully")

In [ ]:
from google.colab import files
files.download("dermatology_cnn_model.h5")